In [ ]:
import sagemaker
import boto3
from sagemaker.model import Model
from sagemaker.model import FrameworkModel
#from sagemaker.pipeline_model import PipelineModel
from sagemaker.pipeline import PipelineModel
from sagemaker import get_execution_role

from time import gmtime, strftime

from sagemaker.predictor import json_serializer, json_deserializer, Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
model_name = 'mock-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [ ]:
ecr_repository_name = "kv"
account_id = role.split(':')[4]
region = boto3.Session().region_name

container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

In [ ]:
#model_data = s3_model_path,
#predictor_cls = sagemaker.predictor.Predictor,


In [ ]:
# missing 2 required positional arguments: 'model_data' and 'entry_point'
    
# model = FrameworkModel(name = model_name,   
#                        image_uri = container_image_uri,
#                        role=role,
#                        env = {
#                            'SAGEMAKER_USE_NGINX': 'true',
#                            'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
#                            'SAGEMAKER_MODEL_SERVER_TIMEOUT': '300'
#                        },
#                        #code_location=s3_code_location,
#                        #sagemaker_session=sagemaker_session #comment this line for local mode.
#                       )



 --- --- Version 1 - working example --- ---

In [ ]:
model = Model(name = model_name,   
                       image_uri = container_image_uri,
                       role=role,
                       env = {
                           'SAGEMAKER_USE_NGINX': 'true',
                           'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
                           'SAGEMAKER_MODEL_SERVER_TIMEOUT': '300'
                       },
                       #code_location=s3_code_location,
                       #sagemaker_session=sagemaker_session #comment this line for local mode.
                      )

In [ ]:
endpoint_name = 'mock-model-ep-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

In [ ]:
pred = model.deploy(initial_instance_count=1,
                    #instance_type='local',
                    instance_type="ml.t2.medium",
                    endpoint_name=endpoint_name)

In [ ]:
payload = {"key": "sme", "value": 10}

In [ ]:
predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sagemaker.Session(),
                      serializer=JSONSerializer(),
                      deserializer=JSONDeserializer(),
#                       content_type='application/json',
#                       accept='application/json'
                     )

In [ ]:
print(predictor.predict(payload))

In [ ]:
# Note:  this works, older version, but not current with lateest version of SageMaker

# predictor = Predictor(endpoint_name=endpoint_name,
#                       sagemaker_session=sagemaker.Session(),
#                       serializer=json_serializer,
#                       deserializer=json_deserializer,
# #                       content_type='application/json',
# #                       accept='application/json'
#                      )

 --- --- Version 2 - working example --- ---

In [ ]:
predictor.delete_endpoint()